# Imports

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency,ttest_1samp

import unittest


# from sklearn.datasets import load_wine

from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression, LinearRegression  # For linear regression
# from sklearn.linear_model import Lasso, Ridge  # For regularized regression
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, AdaBoostClassifier

# #metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_curve, roc_auc_score, mean_squared_error
# from sklearn.datasets import make_classification


# Start

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,y, random_state=0, test_size=0.25)

pipeline = Pipeline( [('imputer', SimpleImputer(strategy='most_frequent')),
                      ('ohe', OneHotEncoder(drop='first', sparse=False))] )

pipeline.fit(x_train[cat_cols])

In [ ]:
np.array_equal()

# End